# **富邦新一代API - Python範例程式碼**

# 安裝

## 若已安裝過 SDK，可**忽略**此部分

### 安裝新一代API Python SDK
此範例使用Windows版本檔案 <br>
***註:*** SDK檔案名稱 (ex. fubon_neo-1.0.3-cp37-abi3-win_amd64.whl) 請根據實際下載.whl名稱修改 <br>
***註2:*** SDK檔案需與本jupyter notebook檔案至於同一資料夾內，或是修改以下指令之SDK檔案路徑

In [ ]:
# Pip install SDK
!pip install --force-reinstall --no-cache fubon_neo-1.0.3-cp37-abi3-win_amd64.whl

# SDK 版本檢視

In [ ]:
import fubon_neo

In [ ]:
fubon_neo.__version__

# 準備

In [ ]:
 # 匯入 SDK Library
from fubon_neo.sdk import FubonSDK, Order
from fubon_neo.constant import TimeInForce, OrderType, PriceType, MarketType, BSAction

In [ ]:
# 連結 API Server
sdk = FubonSDK()

## 登入

In [ ]:
accounts = sdk.login("H123oooooo", "oooooooo", "./H123oooooo.pfx", "oooooooo")   # 登入帳號 輸入:帳號、密碼、憑證路徑、憑證密碼
print(accounts)

# 行情元件

## **建立行情元件連線**

In [ ]:
sdk.init_realtime() # 建立行情元件連線

## 行情 WebSocket 報價

### WebSocket callback functions 設定

In [ ]:
import json
import traceback

subscribe_ids = []  # 訂閱頻道 id 列表

def handle_connect():  # 連線成功 callback
    print("行情連接成功")

def handle_disconnect(code, message):  # 連接斷線 callback
    print(f"行情連接斷線: {code}, {message}")

def handle_message(message): # 處理接收訊息 callback
    try:
        msg = json.loads(message)
        event = msg["event"]
        data = msg["data"]
    
        if event == "pong":
            return
        
        if event == "subscribed":
            id = data["id"]
            
            if id in subscribe_ids:
                print(f"Error: 訂閱 id {id} 已存在列表中")
            else:
                subscribe_ids.append(id)
    
        elif event == "unsubscribed":
            id = data["id"]
            
            try:
                subscribe_ids.remove(id)
            except:
                print(f"Error: 查無此筆訂閱 id 資料, id {id}")
    
        print(f'market data message: {message}')           
        
    except Exception as e:
        handle_error(f'Error parsing JSON: {e}', traceback.format_exc())

def handle_error(error,traceback_info=None):  # 處理程式錯誤訊息 callback
    print(f'market data error: {error}')
    if traceback_info:
        print(f'Traceback:\n{traceback_info}')

stock = sdk.marketdata.websocket_client.stock
stock.on("connect", handle_connect)
stock.on("message", handle_message)
stock.on("disconnect", handle_disconnect)
stock.on("error", handle_error)

stock.connect()  # WebSocket 連線

###### 訂閱商品資料

In [ ]:
# 訂閱股票最新成交資訊
stock.subscribe({ 
        "channel": 'trades', 
        "symbol": '1101',
        "intradayOddLot": True,
        })

In [ ]:
# 訂閱股票最新最佳五檔委買委賣資訊
stock.subscribe({ 
    'channel': 'books', 
    'symbol': '2330'
})

In [ ]:
# 訂閱股票最新指數行情資料
stock.subscribe({ 
    'channel': 'indices', 
    'symbol': 'IR0001'
})

In [ ]:
target_id = "Dn90D2VM2McNXqzJx9gpsz5q8A460Qu2MA"  #欲取消訂閱之頻道編號(id)

result = stock.unsubscribe(
        {
          "id": target_id
        }
)

### 斷開 WebSocket 連線

In [ ]:
stock.disconnect()

## 行情 WebAPI 查詢

In [ ]:
# 建立行情查詢 WebAPI 連線 Object Instance
restStock = sdk.marketdata.rest_client.stock  

### 日內行情

In [ ]:
# 股票或指數列表（依條件查詢）
result = restStock.intraday.tickers(type='EQUITY', exchange="TWSE", market="TSE")
stock_list = ["8467", "9103", "2330"]  # 抽樣查詢之股票 symbols

print(f"資料長度: {len(result['data'])}\n")

for ticker in result["data"]:
    if ticker["symbol"] in stock_list:
        print(ticker)

In [ ]:
# 取得股票資訊 (依股票代碼查詢)
result = restStock.intraday.ticker(symbol='2330')
print(result)

In [ ]:
# 股票即時報價（依代碼查詢）
result = restStock.intraday.quote(symbol="2330")
print(result)

In [ ]:
# 股票價格Ｋ線（依代碼查詢）
result = restStock.intraday.candles(symbol='2330', timeframe=5)
print(result)

In [ ]:
# 股票成交明細（依代碼查詢）
result = restStock.intraday.trades(symbol='2330')
print(result)

In [ ]:
# 股票分價量表（依代碼查詢）
result = restStock.intraday.volumes(symbol='2330')
print(result)

### 行情快照

In [ ]:
# 股票行情快照（依市場別）
result = restStock.snapshot.quotes(market='TSE')
print(result)

In [ ]:
# 股票漲跌幅排行（依市場別）
result = restStock.snapshot.movers(market='TSE', direction='up', change='percent')
print(result)

In [ ]:
# 股票成交量值排行（依市場別）
result = restStock.snapshot.actives(market='TSE', trade='volume')
print(result)

### 歷史行情

In [ ]:
# 取得 1 年內歷史股價（依代碼查詢）
# P.S. 目前分Ｋ無法指定開始日期（from） 與 結束日期（to），一律回傳近五日資料，並且無法選擇 turnover 與 change 的欄位
result = restStock.historical.candles(**{"symbol": "2330", "from": "2023-07-26", "to": "2024-01-30"})
print(result)

In [ ]:
# 取得近 52 週股價數據（依代碼查詢）
result = restStock.historical.stats(symbol="2330")
print(result)

# 交易 (單筆)

### 建立委託單

In [ ]:
# 定義訂單內容
order = Order(
    buy_sell = BSAction.Buy,
    symbol = "2330",
    price = None,
    quantity = 1000, # 股數; 1000為一張
    market_type = MarketType.Common,
    price_type = PriceType.Reference,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = None, # optional field
);

# 列印訂單內容
print(order)

In [ ]:
# 下單
order_reponse = sdk.stock.place_order(accounts.data[0], order)
print(order_reponse)

### 取得委託單結果

In [ ]:
result = sdk.stock.get_order_results(accounts.data[0])
print(f"筆數: {len(result.data)}")
i = 0
for order_result in result.data:
    print(f"第 {i+1} 筆:")
    print(order_result, end="\n\n")
    i += 1

#### 使用範例：使用委託書或流水序號取得特定委託單物件

In [ ]:
def get_order_by_no(orders, order_or_seq_no, use_order_no=True):
    if orders.data is None:
        print(f"查無委託單資訊")
        return None

    for order in orders.data:
        if use_order_no:  # 以委託書號查詢
            if order.order_no is not None and order.order_no == order_or_seq_no:
                # print(f"提取委託單: {order}")
                return order
        else:  # 以委託單流水序號查詢
            if order.seq_no is not None and order.seq_no == order_or_seq_no:
                # print(f"提取委託單: {order}")
                return order

    # 查無委託單
    field_name = "委託書號" if use_order_no else "委託單流水序號"
    print(f"{field_name} {order_or_seq_no} 查無委託單")
    return None

# 使用範例 1: 以委託書號查詢
orders = sdk.stock.get_order_results(accounts.data[0])  # 查詢所有委託單
order = get_order_by_no(orders, "x0003", use_order_no=True)  # x0016 為目標委託書號
print(f"使用範例 1: 結果委託單 {order}\n")

# # 使用範例 2: 以委託單流水序號查詢
# orders = sdk.stock.get_order_results(accounts.data[0])  # 查詢所有委託單
# order2 = get_order_by_no(orders, "00000237234", use_order_no=False)  # 00000237234 為目標委託單流水序號
# print(f"使用範例 2: 結果委託單 {order2}")

### 改價

In [ ]:
# 使用範例：使用委託單號取得欲修改之委託單物件
target_order = None
target_order_number = "l0001"  # 欲查找之委託單號

response = sdk.stock.get_order_results(accounts.data[0])

if response.is_success:
    for order in response.data:
        if order.order_no == target_order_number:  # 取第一個狀態為成功的委託單為例
            target_order = order

    if target_order is not None:
        print(target_order)
    else:
        print(f"單號 {target_order_number}，查無委託單")

else:
    print("查無資料")
    print(f"response: {response}")

In [ ]:
# 單筆改價
target_order = order
modify_price_obj = sdk.stock.make_modify_price_obj(target_order, "542",)  # 改價
response = sdk.stock.modify_price(accounts.data[0], modify_price_obj)  # 送出改價單

# 印出回應
# print(f"目標委託單:\n{target_order}\n")
print(f"修改回覆:\n{response}")

### 改量

In [ ]:
#單筆改量
target_order = order
modify_quantity_obj = sdk.stock.make_modify_quantity_obj(target_order, 1000)  # 改量
response = sdk.stock.modify_quantity(accounts.data[0], modify_quantity_obj)  # 送出改量單

# 印出回應
# print(f"目標委託單:\n{target_order}\n")
print(f"修改回覆:\n{response}")

#### 刪單

In [ ]:
# 函數: 刪單 by 單號
def del_order(order_no, account):
    if not isinstance(order_no, str):
        print(f"請輸入單號字串，例如\"x0001\"")
        return None

    # 取得委託單 object
    target_order = None
    orders = sdk.stock.get_order_results(account)

    for order in orders.data:
        if order.order_no == order_no:
            target_order = order

    if target_order is None:
        print(f"查無目標委託單, 目標委託單號 {order_no}. 帳號:\n{account}")
        return None
    else:
        response = sdk.stock.cancel_order(account, target_order)
        print(f"刪單回報:\n{response}\n\n")
        return response

# 刪單
response = del_order("x0005", accounts.data[0])  # 可修改單號及帳號

if response is None:
    print("刪單函數執行錯誤")

else:
    target_order_number = response.data.order_no
    # 取得新的委託單資訊
    target_order = None
    orders = sdk.stock.get_order_results(accounts.data[0])
    for order in orders.data:
        if order.order_no == target_order_number:
            target_order = order
            
    if target_order is None:
        print(f"查無目標委託單。目標委託單號 {target_order_number}")
    else:
        print(f"新委託單資訊(單筆):\n{target_order}\n\nstatus: {target_order.status}")  # status 30 代表刪單成功

### 查詢歷史委託

In [ ]:
# 歷史委託
response = sdk.stock.order_history(accounts.data[0], "20240313", "20240313")  # 只供查詢兩日內之歷史資料

if response.is_success:
    
    order_history = response.data
    
    print(f"筆數: {len(order_history)}\n")

    i = 0
    for order_his in order_history:
        i += 1
        print(f"第 {i} 筆:\n{order_his}\n")
        
else:
    print("查尋錯誤")
    print(f"response: {response}")

### 查詢歷史成交

In [ ]:
# 歷史成交
result = sdk.stock.filled_history(accounts.data[0], "20240313", "20240313")  # 只供查詢兩日內之歷史資料
print(result)

### 資券配額查詢

In [ ]:
# 資券配額查詢
result = sdk.stock.margin_quota(accounts.data[0], "2330")
print(result)

### 現冲券配額查詢

In [ ]:
# 現冲券配額查詢
result = sdk.stock.daytrade_and_stock_info(accounts.data[0], "2330")
print(result)

# 交易 (批次)

### 建立批次委託單

In [ ]:
# 建立欲委託清單
orders = [
  Order(
    buy_sell = BSAction.Buy,
    symbol = "2881",
    price = None,
    quantity = 2000,
    market_type = MarketType.Common,
    price_type = PriceType.LimitDown,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = "batch1" # optional field
), Order(
    buy_sell = BSAction.Buy,
    symbol = "1101",
    price = None,
    quantity = 1000,
    market_type = MarketType.Common,
    price_type = PriceType.LimitDown,
    time_in_force = TimeInForce.ROD,
    order_type = OrderType.Stock,
    user_def = "batch2" # optional field
) ]

orders

In [ ]:
# 建立批次委託單
result = sdk.stock.batch_place_order(accounts.data[0], orders)
print(result)

### 取得批次委託送單紀錄

***註：*** 此僅為送單紀錄，無交易狀態更新

In [ ]:
# 取得批次委託列表
result = sdk.stock.batch_order_lists(accounts.data[0])
print(result)

### 取得批次委託送單紀錄明細

***註：*** 此僅為送單紀錄，無交易狀態更新

In [ ]:
# 取得批次委託列表
batch_list = sdk.stock.batch_order_lists(accounts.data[0])

# 取得單筆批次委託明細
target_batch_result = batch_list.data[0]  # 單筆批次委託結果
result = sdk.stock.batch_order_detail(accounts.data[0], target_batch_result)

print(result)

### 批次修改

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

#### 批次修改委託價格

In [ ]:
# 建立批次委託修改物件
modify_objects = []

for order in result_orders:
    the_price = order.after_price

    # 設定擬改價格 (自動設定預修改，僅為測試範例)
    if (the_price * 100) % 10 > 0:
        to_be_price = round(the_price + 0.05, 2)
    elif (the_price * 10) % 10 > 0:
        to_be_price = round(the_price + 0.1, 1)
    else:
        to_be_price = round(the_price + 1)

    # 建立改價 obj
    the_modify_price_obj = sdk.stock.make_modify_price_obj(order, str(to_be_price))
    print(f"委託單編號 {order.order_no}, 現價 {the_price}, 擬改價 {to_be_price}")

    # 放入批次列表
    modify_objects.append(the_modify_price_obj)

# 開始批次改價
if len(modify_objects) > 0:
    modified_results = sdk.stock.batch_modify_price(accounts.data[0], modify_objects)
    
    # 處理批次修改委託價格回傳結果
    if modified_results.is_success:
        print("修改成功")
        print(modified_results.data)
    else:
        print("修改失敗", modified_results.message)

else:
    print("改價列表為空白")

#### 批次修改委託數量

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

In [ ]:
# 建立批次委託修改物件
modify_objects = []

for order in result_orders:
    the_qty = order.after_qty

    if the_qty > 1000:
        to_be_qty = the_qty - 1000
        
        # 建立改價 obj
        the_modify_price_obj = sdk.stock.make_modify_quantity_obj(order, int(to_be_qty))
        print(f"委託單編號 {order.order_no}, 現量 {the_qty}, 擬改量 {to_be_qty}")
    
        # 放入批次列表
        modify_objects.append(the_modify_price_obj)

    else:
        print(f"委託單編號 {order.order_no}, 現量 {the_price}, 現量不足，略過")

# 開始批次改量
if len(modify_objects) > 0:
    modified_results = sdk.stock.batch_modify_quantity(accounts.data[0], modify_objects)
    
    # 處理批次修改委託量回傳結果
    if modified_results.is_success:
        print("修改成功")
        print(modified_results.data)
    else:
        print("修改失敗", modified_results.message)

else:
    print("改價列表為空白，出問題了!")

#### 刪除批次委託單

In [ ]:
# 查詢委託單
response = sdk.stock.get_order_results(accounts.data[0])

# 取得此前批次下單(同Timestamp)成功之委託單
result_orders = []

if not response.is_success:
    print(f"委託單查詢失敗, 查詢結果:\n{orders}")

else:  # 委託單查詢成功
    for order in response.data:
        if (order.user_def is not None) and (str(timestamp) in order.user_def):
            result_orders.append(order)

# 列印
print(f"取得委託單結果:\n{result_orders}")

In [ ]:
for order in result_orders:
    print(f"擬刪除之委託單編號 {order.order_no}\n")

# 開始批次刪除
if len(result_orders) > 0:
    cancel_result = sdk.stock.batch_cancel_order(accounts.data[0], result_orders)
    
    # 處理批次刪除回傳結果
    if cancel_result.is_success:
        print("刪除成功")
        print(cancel_result.data)
    else:
        print("刪除失敗", cancel_result.message)

# 帳務

In [ ]:
# 顯示帳號資訊
accounts.data[0]

### 庫存查詢

In [ ]:
# 庫存查詢
result = sdk.accounting.inventories(accounts.data[0])

if result.is_success:
    print(f"資料筆數: {len(result.data)}\n")
    i = 0
    for inv in result.data:
        i += 1
        print(f"第 {i} 筆\n")
        print(f"{inv}\n")
        
else:
    print("查詢失敗")
    print(result)

### 未實現損益

In [ ]:
# 未實現損益
result = sdk.accounting.unrealized_gains_and_loses(accounts.data[0])
print(result)

### 已實現損益

In [ ]:
# 已實現損益
result = sdk.accounting.realized_gains_and_loses(accounts.data[0])
print(result)

### 維持率查詢

In [ ]:
# 維持率
result = sdk.accounting.maintenance(accounts.data[0])
print(result)

### 交割款查詢

In [ ]:
# 交割款
result = sdk.accounting.query_settlement(accounts.data[0],"3d")
print(result)

### 銀行餘額查詢

In [ ]:
# 銀行餘額
result = sdk.accounting.bank_remain(accounts.data[0])
print(result)

# 交易/帳務 主動回報

In [ ]:
import time
import os 

# A callback to receive quote data
def on_order(err, content):
    print("==下單主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_order(on_order) 

# A callback to receive quote data
def on_order_changed(err, content):
    print("==改單主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_order_changed(on_order_changed) 

def on_filled(err, content):
    print("==成交主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_filled(on_filled)

# A callback to receive quote data
def on_event(err, content):
    print("==事件主動回報==")
    print(f"錯誤訊息 {err}")
    print(f"內容 {content}")
    print("========")
    
sdk.set_on_event(on_event) 